In [5]:
import openturns as ot
import openturns.viewer
import openturns.viewer as viewer
from sdp_clip_module_v2_updated import sdp_clip_analytical_function_v2
from sdp_clip_module_v2_printing_updated import sdp_clip_analytical_function_v2_printing
import numpy as np
import pandas as pd
from matplotlib import pylab as plt

ot.Log.Show(ot.Log.NONE)

The goal of this workflow is to develop two modules for the deterministic sizing exercise that needs to be performed, for which the reliability analysis shall be performed to calculate the probability of failure of the SDP clip.
- First, a quantile function shall be developed to calculate the A or B-basis of the probability distribution of the material properties & the limit loads (LL) of the static loading condition distribution.
- Second, a translator function shall be developed to translate the design variables (n_var = 4) obtained from the optimization process in pymoo, to the one usable for as input to the SDP clip analytical function. The translator shall retrieve all the inputs required for the SDP analytical function from the design variables and other variables are set to nominal (bolt location and pattern is fixed, we shall choose a straight line as our bolting line along X-axis)

# Quantile Function

In [6]:
#material property bounds for allowables: bearing allowable for clip is needed only for optimal sizing, and bolt allowables
#for tension and shear
"""LOADS INPUT DISTRIBUTION----------------------------------------------------------"""
#input loads:
Fx = 100000
Fy = 400000

#defined loading bounds:
Fx_bound = (10/100) #%
Fy_bound = (10/100) #%

#defined distributions:
applied_force_distribution = [ot.Uniform(-0.01 + Fx*(1-Fx_bound), 0.01 + Fx*(1+Fx_bound)),  
                              ot.Uniform(-0.01 + Fy*(1-Fy_bound), 0.01 + Fy*(1+Fy_bound))]

"""ALLOWABLES INPUT DISTRIBUTION----------------------------------------------------------"""
#allowable values input:
flat_ext_bearing_allowables_clip_frame_fuselage = np.array([552, 552, 552]).flatten()
flat_ext_shear_allowables_bolt = 655
flat_ext_tension_allowables_bolt = 1100

#defined bounds:
bolt_all_bound = 10/100 #%
bearing_p1p2p3_bound = 10/100 #%

#defined distributions:
bearing_p123_allowables_distribution = [ot.Uniform(i*(1-bearing_p1p2p3_bound), i*(1+bearing_p1p2p3_bound)) for i in flat_ext_bearing_allowables_clip_frame_fuselage.tolist()]
shear_allowables_distribution = [ot.Uniform(i*(1-bolt_all_bound), i*(1+bolt_all_bound)) for i in [flat_ext_shear_allowables_bolt]]
tension_allowables_distribution = [ot.Uniform(i*(1-bolt_all_bound), i*(1+bolt_all_bound)) for i in [flat_ext_tension_allowables_bolt]]

In [7]:
#A-basis = 99%, B-basis = 90%

percentage = 90/100

#give the qualtile calculator a distribution, and the percentage at which it shall calculate the quantile
clip_allowables_distribution = bearing_p123_allowables_distribution[0] #we shall take only clip bearing
bolt_shear_allowables_distribution = shear_allowables_distribution
bolt_tension_allowables_distribution = tension_allowables_distribution

loadX_distribution = applied_force_distribution[0]
loadY_distribution = applied_force_distribution[1]

B_val_clip = clip_allowables_distribution.computeQuantile(percentage)[0]
B_val_bolt_shear = bolt_shear_allowables_distribution[0].computeQuantile(percentage)[0]
B_val_bolt_tension = bolt_tension_allowables_distribution[0].computeQuantile(percentage)[0]

LLX = loadX_distribution.computeQuantile(1-(1e-5))[0]
LLY = loadY_distribution.computeQuantile(1-(1e-5))[0]

print('B-basis of clip bearing allowable = ', np.round(B_val_clip))
print('B-basis of bolt shear allowable = ', np.round(B_val_bolt_shear))
print('B-basis of bolt tension allowable = ', np.round(B_val_bolt_tension))
print("LLX for P: 1e-5 = ", np.round(LLX))
print("LLY for P: 1e-5 = ", np.round(LLY))

#graph1 = allowables_distribution.drawPDF()
#view = viewer.View(graph1)

#graph2 = loadX_distribution.drawPDF()
#view = viewer.View(graph2)

#graph3 = loadY_distribution.drawPDF()
#view = viewer.View(graph3)

B-basis of clip bearing allowable =  596.0
B-basis of bolt shear allowable =  707.0
B-basis of bolt tension allowable =  1188.0
LLX for P: 1e-5 =  110000.0
LLY for P: 1e-5 =  439999.0


So, the above function from openturns allows for the computation of the quantile (x %) from a distribution we supply to it.
Okay, now that we have the qualtile calculator, we shall implement it on our two distributions: loads and material properties.
- loads, to calculate LL
- material properties, to calculate B-basis

What is a B-basis? It is the allowable value above which at least 90% of the population values will fall with 95% confidence levels.

Why B-basis? Because we have an assembly with redundant pathways, a fail-safe design, which means, if a member fails (eg a bolt), the loads get redistributed safely around the adjacent members (other bolts), so we can use B-basis as our allowable strength to size the structure deterministically. (Source: NIU Stress Analysis, Chapter 04)

Finally, above, we have the input block where distributions are defined with their bounds, for loads and material properties. Nothing shall change in this block. And then we have the qualtile calculator. This block calculates the LL from the loading distributions, and the B-basis for the clip bearing allowables only. 

# Translation & Assembly Process

Now, comes the translation process. What we shall develop here is a function or an algorithm that shall allow the retrieval of all the inputs of the SDP clip analytical (that long list!) from the design variable information from the pymoo optimal design process.
- What do we have from pymoo? The design variables: [nfw, nff, thickness, diameter]
- nfw and nff = number of fasteners in the web and foot respectively. We have nfw = NNinp and nff = NNinpfoot in our SDP code.
- thickness is for clip, which is part 1 as per definition, 2 and 3 are frame and fuselage skin resp. So we have tp1 = t 
- diameter is the diameter value of the bolt. Since we shall use all bolts of the same design and material, same bolts on web and foot, we have d = [diameter]*nfw and dfoot = [diameter]*nff

In [8]:
"""INPUT TRANSLATOR:"""
# x comes from pymoo!
x = {'nfw': 5, 'dfw': 4, 'nff': 6, 't': 2.5059073542284596}
e = 10
p = 10
all_frame_br = 552
all_fuselage_br = 552
bolt_K = 14
t_p2, t_p3 = 1, 2
from translator_module import pymoo_2_ot_translator

pymoo_2_ot_translator(x, e, p, LLX, LLY, B_val_clip , 
                      B_val_bolt_shear, B_val_bolt_tension , all_frame_br, all_fuselage_br, bolt_K, t_p2, t_p3)

NNinp =  5 NNinpfoot =  6
SDP_Clip_Analytical_Input = [10,20,30,40,50,10,10,10,10,10,10,20,30,40,50,60,10,10,10,10,10,10,14,14,14,14,14,14,14,14,14,14,14,4,4,4,4,4,4,4,4,4,4,4,2.50591,1,2,35,0,110000,439999,596.16,552,552,707.4,1188,5,6]#58
flat_bolt_location = [10,20,30,40,50,10,10,10,10,10]#10
flat_foot_bolt_location = [10,20,30,40,50,60,10,10,10,10,10,10]#12
flat_bolt_stiffness = [14,14,14,14,14]
flat_foot_bolt_stiffness = [14,14,14,14,14,14]
flat_bolt_diameter = [4,4,4,4,4]
flat_foot_bolt_diameter = [4,4,4,4,4,4]
flat_sdp_clip_thickness_p1 = 2.5059073542284596
flat_frame_thickness_p2 = 1.0
flat_frame_thickness_p3 = 2.0
flat_force_poa = [35,0]
flat_force_applied = [110000,439999]
flat_ext_bearing_allowables_clip_frame_fuselage = [596.16,552,552]
flat_ext_shear_allowables_bolts = 707.4
flat_ext_tension_allowables_bolts = 1188.0
NNinp, NNinpfoot =  5.0 6.0


class=Point name=Unnamed dimension=58 values=[10,20,30,40,50,10,10,10,10,10,10,20,30,40,50,60,10,10,10,10,10,10,14,14,14,14,14,14,14,14,14,14,14,4,4,4,4,4,4,4,4,4,4,4,2.50591,1,2,35,0,110000,439999,596.16,552,552,707.4,1188,5,6]

# END of Development

In [ ]:
def pymoo_2_ot_translator(x, e, p, LLXinp, LLYinp, B_val_clip , B_val_bolt_shear, B_val_bolt_tension , all_frame_br, all_fuselage_br, bolt_K, t_p2, t_p3, verbose = True):
    nfw = x["nfw"]
    nff = x["nff"]
    t = x["t"]
    dfw = x["dfw"]
    #x is a dict

    #NOTE: Also input bolt locations, their stiffnesses, according to the above obtained optimal design values!!!!!!!!!!!!
    """This block takes in the calues of the optimal design variables and builds the SDP clip 
    input for the analytical evaluation"""

    """---------------------------------------------------------------------------------"""
    """--------------------------Clip Foot Input Block Begins---------------------------"""
    """---------------------------------------------------------------------------------"""
    #Foot bolt locations are (x, z) in global coordinates, and (x, y) in foot's local coordinates!!
    foot_bolt_location = np.array([[p*i + e, 10] for i in range(nff)])
    foot_bolt_stiffness = np.array([[bolt_K]*nff]).transpose()


    sdp_clip_thickness_p1 = t
    frame_thickness_p2 = t_p2
    fuselage_thickness_p3 = t_p3



    foot_bolt_location_xi = foot_bolt_location[:, 0] # 1xN row vector
    foot_bolt_location_zi = foot_bolt_location[:, 1]
    #print("foot bolts location: ", foot_bolt_location_xi, foot_bolt_location_zi)
    #print("foot_bolt_stiffness = ", foot_bolt_stiffness)
    """====================================FOOT CG CALCULATIONS============================================="""
    Kixi_foot = []
    Kizi_foot = []
    for i in range(len(foot_bolt_stiffness)):
        Kixi_foot = np.append(Kixi_foot, foot_bolt_location_xi[i]*foot_bolt_stiffness[i])
        #print(foot_bolt_location_xi[i]*foot_bolt_stiffness[i])
    for i in range(len(foot_bolt_stiffness)):
        Kizi_foot = np.append(Kizi_foot, foot_bolt_location_zi[i]*foot_bolt_stiffness[i])
        #print(foot_bolt_location_zi[i]*foot_bolt_stiffness[i])

    xg_foot = Kixi_foot.sum()/foot_bolt_stiffness.sum()
    zg_foot = Kizi_foot.sum()/foot_bolt_stiffness.sum()
    #print("foot CG = ", xg_foot, zg_foot)
    """================================================================================="""
    #"""This cg of foot bolts will not be the input for the applied loading on the clip sub-assembly!"""


    foot_bolt_diameter = np.array([[dfw]*nff]).transpose()

    foot_bolt_loc_vec = np.array(np.hstack((foot_bolt_location_xi, foot_bolt_location_zi)))
    #print("foot_bolt_loc_vec = ", foot_bolt_loc_vec)









    """================================================================================="""
    """--------------------------Clip Web Input Block Begins-------------------------------------"""
    """================================================================================="""
    """clip_min_x, clip_max_x = [0, length]
    clip_min_y, clip_max_y = [0, web_height]"""

    Fx = LLXinp
    Fy = LLYinp

    xF, yF, zF = [xg_foot, 0, zg_foot] #[25, 10] New POA for the clip load Fx, Fy in global coordinates!
    force_poa = np.array([xF, yF])

    force_applied = np.array([Fx, Fy])
    """---------------------------------------------------------------------------------"""
    #print('force_applied = ', force_applied)
    #print('force_poa = ', force_poa, "3D location xF, yF, zF: (", xF, yF, zF, ")")
    """---------------------------------------------------------------------------------"""


    ext_bearing_allowables_clip_frame_fuselage = np.array([B_val_clip, all_frame_br, all_fuselage_br])
    ext_shear_allowables_bolt = np.round(B_val_bolt_shear, 2)
    ext_tension_allowables_bolt = np.round(B_val_bolt_tension, 2)

    #print('ext_shear_allowables_bolt = ', ext_shear_allowables_bolt)
    #print('ext_tension_allowables_bolt = ', ext_tension_allowables_bolt)
    #print('ext_bearing_allowables_clip = ', ext_bearing_allowables_clip_frame_fuselage)

    """---------------------------------------------------------------------------------"""
    """NOTE: all the remaining computations take the len(bolt_stiffness), so keep this input on top!! and you will have all
    solutions automatically calculated!!"""
    bolt_stiffness = np.array([[bolt_K]*nfw]).transpose()
    #print('bolt_stiffness = ', bolt_stiffness)
    """---------------------------------------------------------------------------------"""
    bolt_location = np.array([[p*i + e, 10] for i in range(nfw)])


    """============================================================================="""
    bolt_loc_vec = np.array(np.hstack((bolt_location[:, 0], bolt_location[:, 1])))
    #print('bolt_loc_vec = ', bolt_loc_vec[:len(bolt_stiffness)], bolt_loc_vec[len(bolt_stiffness):])



    """---------------------------------------------------------------------------------"""
    bolt_diameter = np.array([[dfw]*nfw]).transpose()
    """---------------------------------------------------------------------------------"""


    '''4. for each changes i ninput block, also change the fixed values within the function now!!!'''
    """---------------------------------------------------------------------------------"""
    """--------------------------Input Block Ends-------------------------------------"""
    """---------------------------------------------------------------------------------"""


    """-----------------------------------------OT Integration Block------------------------"""
    """Assembled inputs for OT PyFunc"""
    flat_bolt_location = bolt_loc_vec.flatten()
    flat_foot_bolt_location = foot_bolt_loc_vec.flatten()

    #print("x1, y1 = ", flat_bolt_location)
    flat_bolt_stiffness = bolt_stiffness.flatten()
    flat_foot_bolt_stiffness = foot_bolt_stiffness.flatten()

    flat_bolt_diameter = bolt_diameter.flatten()
    flat_foot_bolt_diameter = foot_bolt_diameter.flatten()

    flat_sdp_clip_thickness_p1 = sdp_clip_thickness_p1
    flat_frame_thickness_p2 = frame_thickness_p2
    flat_fuselage_thickness_p3 = fuselage_thickness_p3





    flat_force_poa = force_poa
    flat_force_applied = force_applied

    flat_ext_bearing_allowables_clip_frame_fuselage = ext_bearing_allowables_clip_frame_fuselage
    flat_ext_shear_allowables_bolt = ext_shear_allowables_bolt
    flat_ext_tension_allowables_bolt = ext_tension_allowables_bolt

    """--------------------OT Point ready for implementation----------------------------------"""

    NNinp = x["nfw"]
    NNinpfoot = x["nff"]
    print("NNinp = ", NNinp, "NNinpfoot = ", NNinpfoot)
    """NN takes in bolt qty from the input block"""
    """Big vector generated below!!!!!"""
    SDP_Clip_Analytical_Input_Op = ot.Point(np.hstack((flat_bolt_location, flat_foot_bolt_location,
                                                    flat_bolt_stiffness, flat_foot_bolt_stiffness,
                                                    flat_bolt_diameter, flat_foot_bolt_diameter,
                                                    flat_sdp_clip_thickness_p1,
                                                    flat_frame_thickness_p2,
                                                    flat_fuselage_thickness_p3,
                                                    flat_force_poa,flat_force_applied,
                                                    flat_ext_bearing_allowables_clip_frame_fuselage,
                                                    flat_ext_shear_allowables_bolt, flat_ext_tension_allowables_bolt,
                                                    NNinp, NNinpfoot)))
    if verbose:
        NN = NNinp
        NNfoot = NNinpfoot
        print("SDP_Clip_Analytical_Input =", SDP_Clip_Analytical_Input_Op)
        print("flat_bolt_location =", SDP_Clip_Analytical_Input_Op[0:NN*2])
        print("flat_foot_bolt_location =", SDP_Clip_Analytical_Input_Op[NN*2:(NN*2 + NNfoot*2)])

        print("flat_bolt_stiffness =", SDP_Clip_Analytical_Input_Op[(NN*2 + NNfoot*2):(NN*3 + NNfoot*2)])
        print("flat_foot_bolt_stiffness =", SDP_Clip_Analytical_Input_Op[(NN*3 + NNfoot*2):(NN*3 + NNfoot*3)])

        print("flat_bolt_diameter =", SDP_Clip_Analytical_Input_Op[(NN*3 + NNfoot*3):(NN*4 + NNfoot*3)])
        print("flat_foot_bolt_diameter =", SDP_Clip_Analytical_Input_Op[(NN*4 + NNfoot*3):(NN*4 + NNfoot*4)])

        print("flat_sdp_clip_thickness_p1 =", SDP_Clip_Analytical_Input_Op[(NN*4 + NNfoot*4)])
        print("flat_frame_thickness_p2 =", SDP_Clip_Analytical_Input_Op[(NN*4 + NNfoot*4) + 1])
        print("flat_frame_thickness_p3 =", SDP_Clip_Analytical_Input_Op[(NN*4 + NNfoot*4) + 2])

        print("flat_force_poa =", SDP_Clip_Analytical_Input_Op[(NN*4 + NNfoot*4) + 3:(NN*4 + NNfoot*4) + 5])
        print("flat_force_applied =", SDP_Clip_Analytical_Input_Op[(NN*4 + NNfoot*4) + 5 : (NN*4 + NNfoot*4) + 7])

        print("flat_ext_bearing_allowables_clip_frame_fuselage =", SDP_Clip_Analytical_Input_Op[(NN*4 + NNfoot*4) + 7:(NN*4 + NNfoot*4) + 10])
        print("flat_ext_shear_allowables_bolts =", SDP_Clip_Analytical_Input_Op[(NN*4 + NNfoot*4) + 10])
        print("flat_ext_tension_allowables_bolts =", SDP_Clip_Analytical_Input_Op[(NN*4 + NNfoot*4) + 11])

        print("NNinp, NNinpfoot = ", SDP_Clip_Analytical_Input_Op[-2], SDP_Clip_Analytical_Input_Op[-1])
    return SDP_Clip_Analytical_Input_Op

#we shall input the optimal design values from the out dictionary into the following block to create SDP full list of inputs:
nfw = 4
nff = 3
t = 2.7
dfw = 6.35
x = [nfw, nff, t, dfw]

#NOTE: Also input bolt locations, their stiffnesses, according to the above obtained optimal design values!!!!!!!!!!!!
"""This block takes in the calues of the optimal design variables and builds the SDP clip 
input for the analytical evaluation"""

"""---------------------------------------------------------------------------------"""
"""--------------------------Clip Foot Input Block Begins---------------------------"""
"""---------------------------------------------------------------------------------"""
#Foot bolt locations are (x, z) in global coordinates, and (x, y) in foot's local coordinates!!
foot_bolt_location = np.array([[15, 5], 
                               [25, 5], 
                               [35, 5]])

foot_bolt_stiffness = np.array([[14],
                                [14],
                                [14]])


sdp_clip_thickness_p1 = x[2]
frame_thickness_p2 = 3
fuselage_thickness_p3 = 1



foot_bolt_location_xi = foot_bolt_location[:, 0] # 1xN row vector
foot_bolt_location_zi = foot_bolt_location[:, 1]
#print("foot bolts location: ", foot_bolt_location_xi, foot_bolt_location_zi)
#print("foot_bolt_stiffness = ", foot_bolt_stiffness)
"""====================================FOOT CG CALCULATIONS============================================="""
Kixi_foot = []
Kizi_foot = []
for i in range(len(foot_bolt_stiffness)):
    Kixi_foot = np.append(Kixi_foot, foot_bolt_location_xi[i]*foot_bolt_stiffness[i])
    #print(foot_bolt_location_xi[i]*foot_bolt_stiffness[i])
for i in range(len(foot_bolt_stiffness)):
    Kizi_foot = np.append(Kizi_foot, foot_bolt_location_zi[i]*foot_bolt_stiffness[i])
    #print(foot_bolt_location_zi[i]*foot_bolt_stiffness[i])

xg_foot = Kixi_foot.sum()/foot_bolt_stiffness.sum()
zg_foot = Kizi_foot.sum()/foot_bolt_stiffness.sum()
#print("foot CG = ", xg_foot, zg_foot)
"""================================================================================="""
#"""This cg of foot bolts will not be the input for the applied loading on the clip sub-assembly!"""


foot_bolt_diameter = np.array([x[3]]*x[1])

foot_bolt_loc_vec = np.array(np.hstack((foot_bolt_location_xi, foot_bolt_location_zi)))
#print("foot_bolt_loc_vec = ", foot_bolt_loc_vec)









"""================================================================================="""
"""--------------------------Clip Web Input Block Begins-------------------------------------"""
"""================================================================================="""
clip_min_x, clip_max_x = [0, 50]
clip_min_y, clip_max_y = [0, 20]

Fx = LLX
Fy = LLY

xF, yF, zF = [xg_foot, 0, zg_foot] #[25, 10] New POA for the clip load Fx, Fy in global coordinates!
force_poa = np.array([xF, yF])

force_applied = np.array([Fx, Fy])
"""---------------------------------------------------------------------------------"""
#print('force_applied = ', force_applied)
#print('force_poa = ', force_poa, "3D location xF, yF, zF: (", xF, yF, zF, ")")
"""---------------------------------------------------------------------------------"""


ext_bearing_allowables_clip_frame_fuselage = np.array([B_val_clip, 552, 552])
ext_shear_allowables_bolt = np.round(B_val_bolt_shear, 2)
ext_tension_allowables_bolt = np.round(B_val_bolt_tension, 2)

#print('ext_shear_allowables_bolt = ', ext_shear_allowables_bolt)
#print('ext_tension_allowables_bolt = ', ext_tension_allowables_bolt)
#print('ext_bearing_allowables_clip = ', ext_bearing_allowables_clip_frame_fuselage)

"""---------------------------------------------------------------------------------"""
"""NOTE: all the remaining computations take the len(bolt_stiffness), so keep this input on top!! and you will have all
solutions automatically calculated!!"""
bolt_stiffness = np.array([[12],
                           [12],
                           [12],
                           [12]])
#print('bolt_stiffness = ', bolt_stiffness)
"""---------------------------------------------------------------------------------"""
bolt_location = np.array([[10, 10], #X_Y
                          [20, 10], 
                          [30, 10], 
                          [40, 10]])


"""============================================================================="""
bolt_loc_vec = np.array(np.hstack((bolt_location[:, 0], bolt_location[:, 1])))
#print('bolt_loc_vec = ', bolt_loc_vec[:len(bolt_stiffness)], bolt_loc_vec[len(bolt_stiffness):])



"""---------------------------------------------------------------------------------"""
bolt_diameter = np.array([x[3]]*x[0])
"""---------------------------------------------------------------------------------"""


'''4. for each changes i ninput block, also change the fixed values within the function now!!!'''
"""---------------------------------------------------------------------------------"""
"""--------------------------Input Block Ends-------------------------------------"""
"""---------------------------------------------------------------------------------"""


"""-----------------------------------------OT Integration Block------------------------"""
"""Assembled inputs for OT PyFunc"""
flat_bolt_location = bolt_loc_vec.flatten()
flat_foot_bolt_location = foot_bolt_loc_vec.flatten()

#print("x1, y1 = ", flat_bolt_location)
flat_bolt_stiffness = bolt_stiffness.flatten()
flat_foot_bolt_stiffness = foot_bolt_stiffness.flatten()

flat_bolt_diameter = bolt_diameter.flatten()
flat_foot_bolt_diameter = foot_bolt_diameter.flatten()

flat_sdp_clip_thickness_p1 = sdp_clip_thickness_p1
flat_frame_thickness_p2 = frame_thickness_p2
flat_fuselage_thickness_p3 = fuselage_thickness_p3





flat_force_poa = force_poa
flat_force_applied = force_applied

flat_ext_bearing_allowables_clip_frame_fuselage = ext_bearing_allowables_clip_frame_fuselage
flat_ext_shear_allowables_bolt = ext_shear_allowables_bolt
flat_ext_tension_allowables_bolt = ext_tension_allowables_bolt

"""--------------------OT Point ready for implementation----------------------------------"""

NNinp = x[0]
NNinpfoot = x[1]
print("NNinp = ", NNinp, "NNinpfoot = ", NNinpfoot)
"""NN takes in bolt qty from the input block"""
"""Big vector generated below!!!!!"""
SDP_Clip_Analytical_Input = ot.Point(np.hstack((flat_bolt_location, flat_foot_bolt_location,
                                                flat_bolt_stiffness, flat_foot_bolt_stiffness,
                                                flat_bolt_diameter, flat_foot_bolt_diameter,
                                                flat_sdp_clip_thickness_p1,
                                                flat_frame_thickness_p2,
                                                flat_fuselage_thickness_p3,
                                                flat_force_poa,flat_force_applied,
                                                flat_ext_bearing_allowables_clip_frame_fuselage,
                                                flat_ext_shear_allowables_bolt, flat_ext_tension_allowables_bolt,
                                                NNinp, NNinpfoot)))




colors = 'blue'#np.random.rand(NNinp)
colors_foot = 'red'#np.random.rand(NNinp)
area = (np.pi/4)*(bolt_diameter)**2  # 0 to 15 point radii
area_foot = (np.pi/4)*(foot_bolt_diameter)**2  # 0 to 15 point radii

plt.scatter(bolt_loc_vec[0:len(bolt_stiffness)], bolt_loc_vec[len(bolt_stiffness):], s = area, c=colors, alpha=1)

plt.scatter(foot_bolt_loc_vec[0:len(foot_bolt_stiffness)], [0]*len(foot_bolt_stiffness), s = area_foot, c=colors_foot, alpha=1)

plt.scatter(xF, yF, marker='x', label = 'xg_foot, yg_foot', color="red", s=100)



plt.xlim(clip_min_x, clip_max_x)
plt.ylim(clip_min_y, clip_max_y)
if Fx==0 and Fy!= 0:
    plt.arrow(xF,yF,0, Fy/40000,width=0.2, color = 'green')
elif Fy==0 and Fx != 0:
    plt.arrow(xF,yF,Fx/10000, 0,width=0.2, color = 'm')
else:
    plt.arrow(xF,yF,0, Fy/40000,width=0.2, color = 'green')
    plt.arrow(xF,yF,Fx/10000, 0,width=0.2, color = 'm')



plt.gca().update(dict(title='BOLTING PATTERN', xlabel='x (foot)', ylabel='y (web)'))

# END